# Dynamic Data Masking on Synapse

## Sample Scripts

In [25]:
SELECT c.name, tbl.name as table_name, c.is_masked, c.masking_function  
FROM sys.masked_columns AS c  
JOIN sys.tables AS tbl   
    ON c.[object_id] = tbl.[object_id]  
WHERE is_masked = 1;  

(4 rows affected)

Total execution time: 00:00:00.074

name,table_name,is_masked,masking_function
BirthDate,DimCustomer,1,default()
EmailAddress,DimCustomer,1,email()
BirthDate,DimEmployee,1,default()
EmailAddress,DimEmployee,1,default()


In [26]:
--Clean up masking
ALTER TABLE cso.DimCustomer  
ALTER COLUMN BirthDate DROP MASKED;  

ALTER TABLE cso.DimCustomer  
ALTER COLUMN EmailAddress DROP MASKED;  

Commands completed successfully.

Total execution time: 00:00:00.162

In [27]:
--Add data masking to the DimCustomer Table

ALTER TABLE cso.DimCustomer ALTER COLUMN BirthDate ADD MASKED WITH (FUNCTION = 'default()');

ALTER TABLE cso.DimCustomer ALTER COLUMN EmailAddress ADD MASKED WITH (FUNCTION = 'Email()');



Commands completed successfully.

Total execution time: 00:00:00.128

In [28]:
  
  -- impersonate for testing:
EXECUTE AS USER = 'TestUser';  

SELECT Top 10 CustomerKey, FirstName, LastName, BirthDate, EmailAddress FROM cso.DimCustomer  

REVERT;  


(10 rows affected)

Total execution time: 00:00:00.349

CustomerKey,FirstName,LastName,BirthDate,EmailAddress
5248,Anna,Brown,1900-01-01 00:00:00.000,aXXX@XXXX.com
12143,Cara,Wu,1900-01-01 00:00:00.000,cXXX@XXXX.com
6457,Trevor,Wood,1900-01-01 00:00:00.000,tXXX@XXXX.com
12073,Toni,Srini,1900-01-01 00:00:00.000,tXXX@XXXX.com
6106,Madison,Walker,1900-01-01 00:00:00.000,mXXX@XXXX.com
2977,Emmanuel,Smith,1900-01-01 00:00:00.000,eXXX@XXXX.com
2512,Ernest,Liu,1900-01-01 00:00:00.000,eXXX@XXXX.com
1898,Alyssa,Wilson,1900-01-01 00:00:00.000,aXXX@XXXX.com
9323,Rachel,Wood,1900-01-01 00:00:00.000,rXXX@XXXX.com
7680,Seth,Wood,1900-01-01 00:00:00.000,sXXX@XXXX.com


In [29]:
--Show issues with UNMASK

GRANT UNMASK TO TestUser;

EXECUTE AS USER = 'TestUser';  

SELECT Top 10 CustomerKey, FirstName, LastName, BirthDate, EmailAddress FROM cso.DimCustomer; 
SELECT Top 10 EmployeeKey, FirstName, LastName, BirthDate, EmailAddress FROM cso.DimEmployee;

REVERT;

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:01.383

CustomerKey,FirstName,LastName,BirthDate,EmailAddress
5248,Anna,Brown,1959-05-03 00:00:00.000,anna65@adventure-works.com
12143,Cara,Wu,1956-07-01 00:00:00.000,cara3@adventure-works.com
6457,Trevor,Wood,1954-05-13 00:00:00.000,trevor1@adventure-works.com
12073,Toni,Srini,1940-04-19 00:00:00.000,toni8@adventure-works.com
6106,Madison,Walker,1971-11-26 00:00:00.000,madison23@adventure-works.com
2977,Emmanuel,Smith,1968-05-16 00:00:00.000,emmanuel8@adventure-works.com
2512,Ernest,Liu,1941-05-04 00:00:00.000,ernest4@adventure-works.com
1898,Alyssa,Wilson,1970-01-06 00:00:00.000,alyssa6@adventure-works.com
9323,Rachel,Wood,1963-04-18 00:00:00.000,rachel49@adventure-works.com
7680,Seth,Wood,1972-07-26 00:00:00.000,seth50@adventure-works.com


EmployeeKey,FirstName,LastName,BirthDate,EmailAddress
250,Cindy,White,1942-04-03 00:00:00.000,frank3@contoso.com
126,Grigory,Pogulsky,1967-02-16 00:00:00.000,jeff0@contoso.com
178,Patrick,Sands,1981-05-07 00:00:00.000,steve0@contoso.com
152,Wendy,Richardson,1973-02-07 00:00:00.000,james1@contoso.com
54,Eiji,Yasuda,1975-03-10 00:00:00.000,kirk0@contoso.com
204,David,Simpson,1967-11-27 00:00:00.000,hazem0@contoso.com
230,Iben,Thorell,1960-04-07 00:00:00.000,karan0@contoso.com
81,Ari,Bixhorn,1976-07-06 00:00:00.000,diane1@contoso.com
107,Robert,Brown,1963-02-25 00:00:00.000,lolan0@contoso.com
27,Oleg,Anashkin,1973-11-26 00:00:00.000,zheng0@contoso.com


In [31]:
GRANT UNMASK ON OBJECT::cso.DimCustomer(EmailAddress) TO MarketingUser
GRANT UNMASK ON OBJECT::cso.DimCustomer(BirthDate) TO HRManager
GRANT UNMASK ON SCHEMA::cso TO schemaowner

Commands completed successfully.

Total execution time: 00:00:00.119

In [33]:
--Test Makreting User

EXECUTE AS USER = 'MarketingUser';  

SELECT Top 10 CustomerKey, FirstName, LastName, BirthDate, EmailAddress FROM cso.DimCustomer; 

REVERT;

(10 rows affected)

Total execution time: 00:00:00.380

CustomerKey,FirstName,LastName,BirthDate,EmailAddress
5248,Anna,Brown,1900-01-01 00:00:00.000,aXXX@XXXX.com
12143,Cara,Wu,1900-01-01 00:00:00.000,cXXX@XXXX.com
6457,Trevor,Wood,1900-01-01 00:00:00.000,tXXX@XXXX.com
12073,Toni,Srini,1900-01-01 00:00:00.000,tXXX@XXXX.com
6106,Madison,Walker,1900-01-01 00:00:00.000,mXXX@XXXX.com
2977,Emmanuel,Smith,1900-01-01 00:00:00.000,eXXX@XXXX.com
2512,Ernest,Liu,1900-01-01 00:00:00.000,eXXX@XXXX.com
1898,Alyssa,Wilson,1900-01-01 00:00:00.000,aXXX@XXXX.com
9323,Rachel,Wood,1900-01-01 00:00:00.000,rXXX@XXXX.com
7680,Seth,Wood,1900-01-01 00:00:00.000,sXXX@XXXX.com


In [41]:
--Test HRManager

EXECUTE AS USER = 'HRManager';  

SELECT Top 10 CustomerKey, FirstName, LastName, BirthDate, EmailAddress FROM cso.DimCustomer; 

REVERT;

: Msg 15517, Level 16, State 1, Line 1
Cannot execute as the database principal because the principal "HRManager" does not exist, this type of principal cannot be impersonated, or you do not have permission.

Total execution time: 00:00:00.058